# A notebook to Plot results

### Seaborn

The following is based on the Seaborn tutorial: https://seaborn.pydata.org/tutorial/axis_grids.html

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
sns.set(style="ticks")

#### Read in your results

Select your data file:

In [ ]:
df = pd.read_csv('../data/final/IAEE_residential_results.csv')
df.tail()

#### To plot all economies:

In [ ]:
g = sns.FacetGrid(df, col="Economy", hue="Fuel" ,col_wrap=2, height=5, ylim=(0, 8000))
g.map(plt.plot,"Year","Demand", alpha=0.7)
g.add_legend()

#### To plot one economy in more detail:

In [ ]:
economy = '01_AUS'
to_plot = df[df['Economy']==economy]

In [ ]:
h = sns.FacetGrid(to_plot, col="Fuel", col_wrap=3, height=3.5, ylim=(0, 8000))                                                        
h.map(plt.plot, "Year", "Demand")

### Bokeh

In [ ]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.layouts import gridplot
output_notebook()

Make pivot table so columns are individual lines

In [ ]:
df_pivot = df.pivot_table(index=['Economy','Year'],columns='Fuel',values='Demand')
df_pivot.head(2)

#### Line plot

This will loop through economies.

Clicking on legend item will "mute" it.

In [ ]:
from bokeh.palettes import Spectral10
figures = []

economies = ['01_AUS','18_CT']

for economy in economies:
    q = figure(width=500, height=300,title=economy) 
    _df = df_pivot.loc[(economy,),]
    numlines=len(_df.columns)
    
    for name, color in zip(_df,Spectral10):
        q.line(_df.index.values, _df[name], line_width=2, color=color,
               alpha=0.7, legend_label=name, muted_color=color, muted_alpha=0.2,)
        q.legend.location = "top_left"
        q.legend.click_policy="mute"
    figures.append(q)

show(gridplot(figures, ncols=2, plot_width=400, plot_height=300))

#### Stacked area plot

In [ ]:
from bokeh.palettes import brewer

figures = []

economies = ['01_AUS','18_CT']

for economy in economies:
    _df = df_pivot.loc[(economy,),]
    numlines=len(_df.columns)
    names = _df.columns.values.tolist()
    _df = _df.reset_index()
    v = figure(width=500, height=300,title=economy) 
    v.varea_stack(names, x='Year', 
                  color=brewer['Spectral'][numlines],
                  alpha=0.7,
                  muted_color=brewer['Spectral'][numlines], 
                  muted_alpha=0.2,
                  legend_label=names, 
                  source=_df)
    v.legend.location = "top_left"
    v.legend.click_policy="mute"
    figures.append(v)

show(gridplot(figures, ncols=2, plot_width=400, plot_height=300))